In [1]:
import getpass
import os
import time
from dotenv import load_dotenv

# Load the .env file
_ = load_dotenv()

### Model

In [2]:
from agno.models.ollama import Ollama

sql_ollama_model=Ollama(id="qwen2.5-coder:7b")

## Database Connection

In [3]:
from agno.tools.sql import SQLTools
from sqlalchemy import create_engine

In [4]:
db_url = 'sqlite:///../data/shop.db'
engine = create_engine(db_url)

In [5]:
tool = SQLTools(db_engine=engine)
tool

<SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>

In [6]:
tool.functions

OrderedDict([('list_tables',
              Function(name='list_tables', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.list_tables of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('describe_table',
              Function(name='describe_table', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=None, entrypoint=<bound method SQLTools.describe_table of <SQLTools name=sql_tools functions=['list_tables', 'describe_table', 'run_sql_query']>>, sanitize_arguments=True, show_result=False, stop_after_tool_call=False, pre_hook=None, post_hook=None)),
             ('run_sql_query',
              Function(name='run_sql_query', description=None, parameters={'type': 'object', 'properties': {}, 'required': []}, strict=Non

In [7]:
tool.list_tables()

'["Customers", "OrderDetails", "Orders", "Products"]'

In [8]:
tool.run_sql(sql="SELECT * FROM Customers", limit=2)

[{'CustomerID': 1,
  'FirstName': 'Juan',
  'LastName': 'Pérez',
  'Email': 'juan.perez@email.com',
  'Phone': '555-1234',
  'Address': 'Calle de la Rosa 12, Sevilla'},
 {'CustomerID': 2,
  'FirstName': 'Ana',
  'LastName': 'López',
  'Email': 'ana.lopez@email.com',
  'Phone': '555-5678',
  'Address': 'Av. Libertad 3, Valencia'}]

* Ejecutar la query

In [9]:
data = tool.run_sql(sql="SELECT * FROM Customers", limit=2)
print(data)

import pandas as pd
df = pd.DataFrame(data)
df.head()

[{'CustomerID': 1, 'FirstName': 'Juan', 'LastName': 'Pérez', 'Email': 'juan.perez@email.com', 'Phone': '555-1234', 'Address': 'Calle de la Rosa 12, Sevilla'}, {'CustomerID': 2, 'FirstName': 'Ana', 'LastName': 'López', 'Email': 'ana.lopez@email.com', 'Phone': '555-5678', 'Address': 'Av. Libertad 3, Valencia'}]


,CustomerID,FirstName,LastName,Email,Phone,Address
0,1,Juan,Pérez,juan.perez@email.com,555-1234,"Calle de la Rosa 12, Sevilla"
1,2,Ana,López,ana.lopez@email.com,555-5678,"Av. Libertad 3, Valencia"


In [11]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>CustomerID</th>\n      <th>FirstName</th>\n      <th>LastName</th>\n      <th>Email</th>\n      <th>Phone</th>\n      <th>Address</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>1</td>\n      <td>Juan</td>\n      <td>Pérez</td>\n      <td>juan.perez@email.com</td>\n      <td>555-1234</td>\n      <td>Calle de la Rosa 12, Sevilla</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>2</td>\n      <td>Ana</td>\n      <td>López</td>\n      <td>ana.lopez@email.com</td>\n      <td>555-5678</td>\n      <td>Av. Libertad 3, Valencia</td>\n    </tr>\n  </tbody>\n</table>'

### QuerySql agent

In [ ]:
# from agno.agent import Agent

# No funciona con el modelo de ollama
# querysql_agent = Agent(model=sql_ollama_model, name="sqlagent", tools=[tool.run_sql], show_tool_calls=True)

In [15]:
import os
from pathlib import Path
from dotenv import load_dotenv
from typing import Annotated, Literal, Any, Optional, List
from dataclasses import dataclass
from agno.models.ollama import Ollama
from ollama import Client as OllamaClient
from agno.agent import Agent
from sql_agent.prompt import TEXT2SQL_TEMPLATE
from sql_agent.utils import logger
import pandas as pd

log = logger.get_logger(__name__)


@dataclass
class Text2SQLAgent:
	""" Text to SQL Agent to convert natural language to SQL """	
	def __post_init__(self,	db_url = 'sqlite:///../data/shop.db'):
		# Instantiate the Ollama model
		self.ollama_model= Ollama(id="qwen2.5-coder:7b")

		self.agent_name = "text2sql"

		# Instantiate the agent
		self.text2sql_agent = Agent(model=self.ollama_model, 
								name = self.agent_name, 
								role="Text to SQL Agent",
								instructions=[TEXT2SQL_TEMPLATE], 
								show_tool_calls=True)
		
		# Instantiate the SQLTools
		self.engine = create_engine(db_url)
		self.tools = SQLTools(db_engine=self.engine)

	def write_query(self, question:str):
		""" Run the agent and return the sql query
		Args:
			question (str): question
		Returns:
			sql_query (str): sql query
		"""
		response = self.text2sql_agent.run(question)
		sql_query = response.content
		return sql_query

	def execute_query(self, sql_query:str):
		""" Run the sql query 
		Args:
			sql_query (str): sql query
		Returns:
			results: Results
		"""
		data = self.tools.run_sql(sql=sql_query)
		df = pd.DataFrame(data)
		if df.empty:
			return "No hay datos"
		else:
			return df.to_html()

	def request(self, question):
		""" Run the agent and return the answer. """
		
		log.info(f"Writing sql query for the question:{question}")
		sql_query = self.write_query(question)
		log.info(f"Executing the sql query:{sql_query}")
		answer = self.execute_query(sql_query)
		return sql_query,answer

In [16]:
text2sql_agent = Text2SQLAgent()

In [17]:
query = "¿Cuáles son los detalles de todos los clientes?"
sql_query,answer = text2sql_agent.request(query)

print(sql_query)
print(answer)

SELECT CustomerID, FirstName, LastName, Email, Phone, Address FROM Customers;
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>CustomerID</th>
      <th>FirstName</th>
      <th>LastName</th>
      <th>Email</th>
      <th>Phone</th>
      <th>Address</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>1</td>
      <td>Juan</td>
      <td>Pérez</td>
      <td>juan.perez@email.com</td>
      <td>555-1234</td>
      <td>Calle de la Rosa 12, Sevilla</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2</td>
      <td>Ana</td>
      <td>López</td>
      <td>ana.lopez@email.com</td>
      <td>555-5678</td>
      <td>Av. Libertad 3, Valencia</td>
    </tr>
    <tr>
      <th>2</th>
      <td>3</td>
      <td>Carlos</td>
      <td>González</td>
      <td>carlos.gonzalez@email.com</td>
      <td>555-8765</td>
      <td>Calle del Sol 7, Madrid</td>
    </tr>
    <tr>
      <th>3</th>
      <td>4</td>
      <td>Laura<

### Text 2 sql agent

In [3]:
from agno.agent import Agent

instructions="""
You are an expert in SQL. You can create sql queries from natural language following these schemas:

CREATE TABLE IF NOT EXISTS Customers(
        CustomerID INTEGER PRIMARY KEY AUTOINCREMENT,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Email TEXT UNIQUE NOT NULL,
        Phone TEXT,
        Address TEXT
);

-- Create Products table
CREATE TABLE IF NOT EXISTS Products (
        ProductID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name TEXT NOT NULL,
        Description TEXT,
        Price REAL NOT NULL,
        StockQuantity INTEGER NOT NULL,
        Category TEXT NOT NULL
    );


-- Create Orders table
CREATE TABLE IF NOT EXISTS Orders (
        OrderID INTEGER PRIMARY KEY AUTOINCREMENT,
        CustomerID INTEGER NOT NULL,
        OrderDate DATE NOT NULL,
        TotalAmount REAL NOT NULL,
        FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
    );

-- Create OrderDetails table
CREATE TABLE IF NOT EXISTS OrderDetails (
        OrderDetailID INTEGER PRIMARY KEY AUTOINCREMENT,
        OrderID INTEGER NOT NULL,
        ProductID INTEGER NOT NULL,
        Quantity INTEGER NOT NULL,
        Subtotal REAL NOT NULL,
        FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
        FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
    );

Return only the sql query. Don't explain the query and don't put the sql query in ```sql\n```.
"""
text2sql_agent = Agent(model=sql_ollama_model, name="text2sql", instructions=instructions, show_tool_calls=True)

### Tests

In [4]:
def write_sql_query(query:str):
    """ Function to write sql query """
    start_time = time.perf_counter()

    response = text2sql_agent.run(query)

    end_time = time.perf_counter()
    
    latency = end_time - start_time
    
    print(f"Execution time: {latency} seconds")
    answer = response.content
    
    return answer
    

In [5]:
query = "¿Cuáles son los detalles de todos los clientes?"
write_sql_query(query) #SELECT * FROM Customers;

Execution time: 109.26334430000861 seconds


'SELECT * FROM Customers;'

In [6]:
query = "¿Cuáles son los productos disponibles en la categoría 'Women'?"
write_sql_query(query)#"SELECT * FROM Products WHERE Category = 'Women';",

Execution time: 6.971404300013091 seconds


"SELECT Name FROM Products WHERE Category = 'Women'"

In [7]:
query = "¿Cuál es el precio total de todos los pedidos realizados por un cliente específico con ID 5?"
write_sql_query(query)#"SELECT SUM(TotalAmount) FROM Orders WHERE CustomerID = 5;",

Execution time: 11.981810800003586 seconds


'SELECT SUM(TotalAmount) FROM Orders WHERE CustomerID = 5'

In [8]:
query = "¿Cuántos productos están en stock con una cantidad mayor a 20 unidades?"
write_sql_query(query)#"SELECT * FROM Products WHERE StockQuantity > 20;"

Execution time: 10.258081999985734 seconds


'SELECT COUNT(*) FROM Products WHERE StockQuantity > 20'

In [9]:
query = "¿Qué productos han sido comprados en el pedido con ID 3?"
write_sql_query(query)#"SELECT p.Name, od.Quantity, od.Subtotal FROM OrderDetails od INNER JOIN Products p ON od.ProductID = p.ProductID WHERE od.OrderID = 3;",

Execution time: 14.596958100009942 seconds


'SELECT P.ProductID, P.Name FROM Products AS P JOIN OrderDetails AS OD ON P.ProductID = OD.ProductID WHERE OD.OrderID = 3;'